# Portfolio Optimization
This notebook demonstrates portfolio optimization techniques on portfolios of N assets. Bootstrapping will be used to estimate the standard errors of the allocation optimization functions. A simple example of two assets is first used to conceptualize the optimization techniques. Once demonstrated, portfolio optimization will be scalled to N assets.

In [1]:
# Notebook imports
import yfinance as yf

### 1. Minimum Variance
Assets yield returns. Suppose we are a risk-averse and would like to allocate our money between two assets with returns X and Y, where X and Y are random continuous variables. We want to determine the allocation that minimizes total portfolio risk (variance). $\alpha$ denotes the allocation to X and $1-\alpha$ denotes the allocation to Y. The total portfolio is then represented as the equation $\alpha*X + (1-\alpha)*Y$. The variance of the portfolio is:<br><br>
$Var(\alpha*X + (1-\alpha)*Y)$<br><br>
This is transformed into a minimization problem.

In [ ]:
X = 'SPY' # S&P 500 Index
Y = 'QQQ' # NASDAQ-100 Index

# Download monthly data
data = yf.download(tickers=[X, Y], interval='1mo', period='max', auto_adjust=True)['Close']

# Calculate the monthly returns
data[X] = data[X].pct_change()
data[Y] = data[Y].pct_change()

# Clean the data
data.dropna(inplace=True)

# Display the summary statistics of the returns
print(data.describe())

[*********************100%***********************]  2 of 2 completed

Ticker         QQQ         SPY
count   314.000000  314.000000
mean      0.009935    0.007170
std       0.067657    0.044359
min      -0.262053   -0.160355
25%      -0.023946   -0.016731
50%       0.014620    0.012796
75%       0.050064    0.033315
max       0.243197    0.133610
